In [137]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

# ~~Наконец-то~~ Перебираем модели

Во-первых, определим метрику

In [138]:
from sklearn.metrics import mean_squared_log_error as MSLE

def RMSLE(y_true, y_pred):
    return np.sqrt(MSLE(y_true, y_pred))

Во-вторых, определим небольшой класс, который призван записывать информацию о валидации, дабы, как минимум, в будущем не потерять хорошую модель

In [139]:
import sys
import math

class Logger:
    def __init__(self, estimator = 'XGBClassifier', eda_info = 'Нет', 
                 val_info = '10 блоков, без стандартизации, без PCA',
                 grid = None, best_params = None, comp_info = 'Сравнивались средние значения',
                 scores = None, outfile = r'logs\log_1.txt'):
        
        '''
        
        Параметры:
            estimator (str): Название модели
            
            eda_info (str): Информация о предобработке данных. Например, 'предварительно с помощью некоторого
                            алгоритма были удалены выбросы'
            
            val_info (str): Информация о валидации: кол-во блоков, использование стандартизации и т.д
            
            grid (dict): Сетка, по которой перебирались гиперпараметры 
            
            best_params (dict): Набор параметров, показавший себя лучше остальных 
            
            comp_info (str): Информация о том, как в процессе валидации сравнивались модели с разными параметрами
            
            scores (numpy.ndarray, tuple): Набор значений метрики лучшей модели по время валидации или кортеж вида
                                         (среднее значение, стандартное отклонение)
            
            outfile (str): Путь к файлу, в который вся информация будет записана 
        
        Возвращаемое значение:
            Объект класса 
        
        '''
        
        self.estimator = estimator
        self.eda_info = eda_info
        self.val_info = val_info
        self.grid = grid
        self.best_params = best_params
        self.comp_info = comp_info
        self.scores = scores
        self.outfile = outfile
        
        # Частенько сетка состоит из одной точки, т.е просто проводится валидация с помощью функции
        # для поиска по сетке, ибо я ленивая жопа и не написал функцию просто для валидации
        # (что даже справедливо, ибо зачем)
        # В общем, в таком случае будет лишним выводить и сетку, и лучшие параметры
        # Так что заводим индикатор того, что сетка состоит из одной точки
                               # Значения - это списки
        self.only_validation = ( [type(i) for i in self.grid.values()] == [list] * len(self.grid) 
                                 and (math.prod([len(i) for i in self.grid.values()]) == 1) )
        
    def scores_info(self, num_of_tabs = 0):
        '''Формирует информацию о метрике лучшей модели
        
        Параметры:
            num_of_tabs (int): Количество табуляций перед каждой строкой
        
        Возвращаемое значение:
            Строкa с информацией о метрике 
        
        '''
        tabs = '\t' * num_of_tabs
        info = ''
        if type(self.scores) == np.ndarray:
            info += f'{tabs}Точность: {self.scores.mean()} +- {self.scores.std()} (mean +- std)\n'
            info += f'{tabs}Худшая точность: {self.scores.max()}\n'
            info += f'{tabs}Медианная точность: {np.median(self.scores)}\n'
            info += f'{tabs}Лучшая точность: {self.scores.min()}\n'
        else:
            info += f'{tabs}Точность: {self.scores[0]} +- {self.scores[1]} (mean +- std)\n'

        return info
    
    def print_grid(self, stream = sys.stdout):
        '''Выводит информацию о сетке гиперпараметров в указанный поток'''
        stream.write('Сетка:\n')
        for key, value in self.grid.items():
            stream.write(f'\t{key}: {value}\n')
                
    def print_best_params(self, stream = sys.stdout):
        '''Выводит информацию о лучшем наборе гиперпараметров в указанный поток'''
        stream.write('Лучший набор параметров:\n')
        if self.only_validation:
            stream.write('\tСетка состоит из одной точки\n')
            return
        
        for key, value in self.best_params.items():
            stream.write(f'\t{key}: {value}\n')
    
    def make_note(self, print_scores_info = True):
        '''Записывает информацию в файл
        
        Параметры:
            print_scores_info (bool): Если True, вывод информацию о лучшей модели ещё и в sys.stdout
                                      Что (не) происходит в противном случае догадаться не сложно
        
        '''
        with open(self.outfile, 'a') as out:
            out.write(f'Модель:\n\t{self.estimator}\n')
            out.write(f'Особенности предобработки данных:\n\t{self.eda_info}\n')
            out.write(f'Особенности валидации:\n\t{self.val_info}\n')
            self.print_grid(stream = out)
            self.print_best_params(stream = out)
            out.write(f'Информация о методе сравнения моделей:\n\t{self.comp_info}\n')
            out.write(f'Информация о метрике:\n{self.scores_info(num_of_tabs = 1)}\n')
            out.write('\n\n' + '*' * 75 + '\n\n')
        
        if print_scores_info:
            print(self.scores_info())

В-третьих, в силу моего маниакального нежелания стандартизовать всю выборку разом, полезно иметь функцию, которая всё стандартизует, обучит модель, получить предсказания и "разстандартизует" всё как было

In [140]:
def teach_and_predict(X_train, X_test, real_features, model, **params):
    '''
    Обучает модель model на X_train, делает предсказания на X_test
    
    Параметры: 
        X_train (pandas.DataFrame): Таблица для обучения

        y_train (pandas.Series): Список значений целевой переменной для объектов из X_train

        X_test (pandas.DataFrame): Таблица, для которой нужно предсказать целевую переменную

        features_to_standard (list, tuple, pandas.Index): Список признаков, которые подлежат стандартизации

        model: Что угодно, что имеет методы fit и predict 

        **params (dict): Словарь, в котором ключи - это названия гиперпараметров модели, а значения - значения 
                         гиперпараметров, извиняюсь за тавтологию
                         
    Возвращаемое значение:
        Кортеж из двух элементов. Первый из них - обученная модель, второй - предсказания модели
        для объектов из X_test
    '''
    train_mean = X_train[real_features].mean()
    train_std = X_train[real_features].std()
    X_train[real_features] = (X_train[real_features] - train_mean) / train_std

    sp_mean = X_train['SalePrice'].mean()
    sp_std = X_train['SalePrice'].std()
    X_train['SalePrice'] = (X_train['SalePrice'] - sp_mean) / sp_std

    test_mean = X_test[real_features].mean()
    test_std = X_test[real_features].std()
    X_test[real_features] = (X_test[real_features] - test_mean) / test_std

    model = model(**params)
    model.fit(X_train.drop('SalePrice', axis = 1), X_train['SalePrice'])

    sp_pred = model.predict(X_test)
    sp_pred = np.exp(sp_pred * sp_std + sp_mean)

    X_train[real_features]  = X_train[real_features] * train_std + train_mean
    X_train['SalePrice'] = X_train['SalePrice'] * sp_std + sp_mean
    X_test[real_features] = X_test[real_features] * test_std + test_mean
    
    # модель возвращается на случай, если захочется "залезть" к ней вовнутрь 
    return model, sp_pred

В-третьих, пригодится функция для стандартизации обучающих и тестовых блоков по отдельности, которая после получения значения метрики вернёт данные к изначальному масштабу

In [141]:
from sklearn.model_selection import KFold

def teach_and_predict_while_grid_search(X, y, train_index, test_index,
                                        real_features, model, **params):
    '''
    Стандартизует отдельно обучающие блоки и отдельно тестовый блок. После обучает указанную модель 
    с указанными параметрами, считает скор и возвращает данные к исходному масштабу. По сути
    то же, что и teach_and_predict, но принимает данные в удобном во время поиска по сетке
    
    Параметры: 
        X (pandas.DataFrame): Таблица с данными (без целевой переменной)

        y (pandas.Series): Список значений целевой переменной
        
        train_index (list, numpy.ndarray, tuple): Список индексов тренировочного набора 

        test_index (list, numpy.ndarray): Список индексов тестового набора
        
        real_features (list, tuple, pandas.Index): Список признаков, которые подлежат стандартизации
        
        model: Что угодно, что имеет методы fit и predict 
        
        **params (dict): Словарь, в котором ключи - это названия гиперпараметров модели, а значения - значения 
                         гиперпараметров. 
    '''
    # Стандартизуем обучающие блоки
    train_mean = X.iloc[train_index][real_features].mean(axis = 0)
    train_std = X.iloc[train_index][real_features].std(axis = 0)
    X.loc[train_index, real_features] = (X.loc[train_index, real_features] - train_mean) / train_std
    
    # Стардартизуем тестовые блоки
    test_mean = X.iloc[test_index][real_features].mean(axis = 0)
    test_std = X.iloc[test_index][real_features].std(axis = 0)
    X.loc[test_index, real_features] = (X.loc[test_index, real_features] - test_mean) / test_std

    # Стандартизуем цены обучающего набора
    train_sp_mean = y.loc[train_index].mean(axis = 0)
    train_sp_std = y.loc[train_index].std(axis = 0)
    y.loc[train_index] = (y.loc[train_index] - train_sp_mean) / train_sp_std

    # Стандартизуем цены тестового набор
    test_sp_mean = y.iloc[test_index].mean(axis = 0)
    test_sp_std = y.iloc[test_index].std(axis = 0)
    y.iloc[test_index] = (y.iloc[test_index] - test_sp_mean) / test_sp_std
    
    # Инициализируем модель
    model = model(**params)
    # Обучаем модель
    model.fit(X.iloc[train_index], y.iloc[train_index])
    
    # Получаем предсказание модели на тестовой выборке
    y_pred = model.predict(X.iloc[test_index])
    # Масштабируем предсказания модели
    y_pred = y_pred * test_sp_std[0] + test_sp_mean[0]
    # Модели предсказывают логарифм цены
    y_pred = np.exp(y_pred)
    
    # Возвращаем тестовые цены на место
    y.iloc[train_index] = y.iloc[train_index] * train_sp_std + train_sp_mean
    # Возвращаем на место цены тестового набора
    y.iloc[test_index] = y.iloc[test_index] * test_sp_std + test_sp_mean

    # Возвращаем на место обучащий блок
    X.loc[train_index, real_features] = X.loc[train_index, real_features] * train_std + train_mean
    # И тестовый тоже 
    X.loc[test_index, real_features] = X.loc[test_index, real_features] * test_std + test_mean
    
    return RMSLE(np.exp(y.iloc[test_index]), y_pred)

Ну и в силу необходимости (а точнее, моего желания) стандартизировать блоки отдельно, придётся изобретать свой ~~велосипед~~ поиск по сетке. В этом, на самом деле, есть один заметный плюс: можно прикрутить tqdm, чтоб быть уверенным хотя бы в том, что перебираться всё это дело будет не до второго пришествия. Иными словами, даже если бы необходимости изобретать этот велосипед не было, его стоило бы изобрести

In [142]:
import math
import itertools
# названия лучше не придумал
def kosherGridSearchCV(X, y, model, param_grid, 
                       features_to_standard, comparator, rand_state,
                       n_splits = 10):
    '''
    Проводит поиск лучших гиперпараметров по указанной сетке. 
    
    Параметры: 
        X (pandas.DataFrame): Таблица с данными (без целевой переменной)

        y (pandas.Series): Список значений целевой переменной
        
        model: Что угодно, что имеет методы fit и predict 
        
        param_grid (dict): Словарь, описывающий все возможные гиперпараметры

        features_to_standard (list, tuple, pandas.Index): Список признаков, которые подлежат стандартизации
        
        comparator (function): Функция, сравнивающая два массива. Должна возвращать отрицательное число,
                               если первый список лучше второго в смысле выбранной метрики 
                               
        rand_state (int): Ключ генератора случайных чисел для sklearn.KFold
        
        n_splits (int): Количество блоков, по которым будет проходить кросс-валидация
    
    Возвращаемое значение:
        Кортеж из двух элементов. Первый из них - это словарь лучших гиперпараметров, 
        второй - значения точности модели на валидации
    '''
    cross_val = KFold(n_splits = n_splits, shuffle = True, random_state = rand_state)

    best_params = {}
    best_scores = np.array([1000] * n_splits) # Массив скоров, хуже которого быть не может

    # общее число возможных комбинаций гиперпараметров
    n_combs = math.prod([len(i) for i in param_grid.values()])
    with tqdm(total = n_combs) as psbar: 
        for comb in itertools.product(*param_grid.values()):
            cur_scores = np.array([])
            # Получаем текущую комбинацию гиперпараметров
            cur_params = { list(param_grid.keys())[i]: comb[i] for i in range(len(comb)) }
            psbar.set_description(f'Processing {cur_params}')

            for train_index, test_index in cross_val.split(X):
                score = teach_and_predict_while_grid_search(X, y, train_index, test_index, features_to_standard,
                                                            model, **cur_params)
                cur_scores = np.append(cur_scores, score)

            if comparator(cur_scores, best_scores) < 0:
                best_scores = cur_scores
                best_params = cur_params

            psbar.update(1)

    return best_params, best_scores

In [143]:
def write_in_file(predictions, filename):
    '''
    Записывает предсказания в отдельный файл 
    '''
    predictions = pd.DataFrame({'Id': [1461 + i for i in range(len(predictions))], 'SalePrice': predictions})
    predictions.set_index('Id', inplace = True)
    predictions.to_csv(filename)

Некоторые методы сравнения значений метрики

In [144]:
# для массивов с одинаковым средним вернёт 1, а не ноль, но малины это не портит
def compare_means(a, b):
    return -1 if a.mean() < b.mean() else 1

def compare_diff_mean(a, b):
    return (a - b).mean()

Загружаем обработанные данные

In [145]:
train_df_x = pd.read_csv(r'processed_data\train_df_x.csv')
train_df_y = pd.read_csv(r'processed_data\train_df_y.csv')
test_df = pd.read_csv(r'processed_data\test_df.csv')

import pickle 
with open(r'processed_data\features_to_standard.pickle', 'rb') as f:
    features_to_standard = pickle.load(f)

In [146]:
train_df = pd.concat([train_df_x, train_df_y], axis = 1)

По непонятным мне причинам, после сохранения DataFrame'ов в файл, появляется вот такой вот странный столбец

In [147]:
train_df_x.drop('Unnamed: 0', axis = 1, inplace = True)
train_df_y.drop('Unnamed: 0', axis = 1, inplace = True)
train_df.drop('Unnamed: 0', axis = 1, inplace = True)
test_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [148]:
EDA_INFO = 'Бейзлайн'

Ну и последняя ремарочка: в роли отложенной выборки выступает его величество kaggle

# Lasso Regression

In [67]:
from sklearn.linear_model import Lasso 

# Пандас истерически (и ложно) орёт, что я пытаюсь присвоить значение копии, а не представлению,
# чем засоряет вывод 
import warnings
warnings.filterwarnings("ignore")

lasso_param_grid = {'alpha': [0, 1e-2, 1e-3, 1e-4, 0.1, 0.5, 1, 2, 5, 10, 15, 20, 25, 50],
                    'selection' : ['cyclic', 'random'],
                    'fit_intercept': [False],
                    'tol': [1e-3, 1e-4, 1e-5],
                    'random_state': [42]}

lasso_log = Logger(estimator = 'Lasso Regression',
                   eda_info = EDA_INFO,
                   val_info = '10 блоков, со стандартизацией и PCA', 
                   grid = lasso_param_grid, outfile = r'logs\linear_models.txt')

lasso_log.best_params, lasso_log.scores = kosherGridSearchCV(train_df_x, train_df_y, Lasso,
                                                             lasso_param_grid, features_to_standard,
                                                             compare_means, 42, 10)

lasso_log.make_note()

  0%|          | 0/84 [00:00<?, ?it/s]

Точность: 0.1317937560538709 +- 0.03629776491038654 (mean +- std)
Худшая точность: 0.2325921659131279
Медианная точность: 0.12493040643596987
Лучшая точность: 0.08991501134870747



In [71]:
lasso_model, lasso_pred = teach_and_predict(train_df, test_df, features_to_standard, Lasso, **lasso_log.best_params)

write_in_file(lasso_pred, r'predictions\lasso_preds.csv')

Скор на отложенной выборке - 0.13633

# Ridge Regression

In [68]:
from sklearn.linear_model import Ridge

ridge_param_grid = {'alpha': [0.1, 0.5, 1, 2, 5, 10],
                    'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'], # запихнул всё шо было
                    'fit_intercept': [False],                                                   # в документации
                    'tol': [1e-3, 1e-4, 1e-5],                    
                    'random_state': [42]}

ridge_log = Logger(estimator = 'Ridge Regression',
                   eda_info = EDA_INFO,
                   val_info = '10 блоков, со стандартизацией и PCA', 
                   grid = ridge_param_grid, outfile = r'logs\linear_models.txt')

ridge_log.best_params, ridge_log.scores = kosherGridSearchCV(train_df_x, train_df_y, Ridge,
                                                             ridge_param_grid, features_to_standard,
                                                             compare_means, 42, 10)

ridge_log.make_note()

  0%|          | 0/126 [00:00<?, ?it/s]

Точность: 0.13193543293520127 +- 0.03641122148624169 (mean +- std)
Худшая точность: 0.23206353531799068
Медианная точность: 0.12262096860856261
Лучшая точность: 0.08931316252250326



In [72]:
ridge_model, ridge_pred = teach_and_predict(train_df, test_df, features_to_standard, Ridge, **ridge_log.best_params)

write_in_file(ridge_pred, r'predictions\ridge_preds.csv')

Весьма и весьма. Скор на отложенной выборке - 0.13312

# ElasticNet

In [69]:
from sklearn.linear_model import ElasticNet

elnet_param_grid = {'alpha': [0.05, 0.1, 0.2],
                    'l1_ratio': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'fit_intercept': [False],
                    'tol': [1e-3, 1e-4, 1e-5],
                    'selection': ['cyclic', 'random'],
                    'positive': [True, False]}

elnet_log = Logger(estimator = 'Elnet Regression',
                   eda_info = EDA_INFO,
                   val_info = '10 блоков, со стандартизацией и PCA', 
                   grid = elnet_param_grid, outfile = r'logs\linear_models.txt')

elnet_log.best_params, elnet_log.scores = kosherGridSearchCV(train_df_x, train_df_y, ElasticNet,
                                                             elnet_param_grid, features_to_standard,
                                                             compare_means, 42, 10)

elnet_log.make_note()

  0%|          | 0/396 [00:00<?, ?it/s]

Точность: 0.13550165367055583 +- 0.03623419899809722 (mean +- std)
Худшая точность: 0.2352251156265658
Медианная точность: 0.1230004366864054
Лучшая точность: 0.09397054232461638



In [74]:
elnet_model, elnet_pred = teach_and_predict(train_df, test_df, features_to_standard, ElasticNet, **elnet_log.best_params)

write_in_file(elnet_pred, 'predictions\elnet_preds.csv')

Скор на отложенной выборке - 0.14398

## SVR

In [75]:
from sklearn.svm import SVR

svr_param_grid = {'kernel': ['rbf'],
                  'gamma': [1/50, 1/75, 1/100, 1/125, 1/150],
                  'C': [1.5, 2.0, 2.5, 5, 10],
                  'epsilon': [0.001, 0.005, 0.025, 0.05, 0.75, 1]}

svr_log = Logger(estimator = 'SVR Regression',
                   eda_info = EDA_INFO,
                   val_info = '10 блоков, со стандартизацией и PCA', 
                   grid = svr_param_grid, outfile = r'logs\linear_models.txt')

svr_log.best_params, svr_log.scores = kosherGridSearchCV(train_df_x, train_df_y, SVR,
                                                         svr_param_grid, features_to_standard,
                                                         compare_means, 42, 10)

svr_log.make_note()

  0%|          | 0/150 [00:00<?, ?it/s]

Точность: 0.12589469667207256 +- 0.02592604722896134 (mean +- std)
Худшая точность: 0.1934920766785683
Медианная точность: 0.11966529093955239
Лучшая точность: 0.09852807857039018



In [76]:
svr_model, svr_pred = teach_and_predict(train_df, test_df, features_to_standard, SVR, **svr_log.best_params)

write_in_file(svr_pred, 'predictions\svr_preds.csv')

Весьма и весьма неплохо. Скор на отложенной выборке - 0.13321

**XGBoost**

Пошла тяжёлая артиллерия

In [122]:
from xgboost import XGBRegressor

xgb_param_grid = {'booster': ['dart', 'gbtree'],
                  'verbosity': [0],
                  'max_depth': [6],
                  'eta': [0.03],
                  'n_estimators': [200, 500],
                  'subsample': [0.3, 0.4],
                  'colsample_bytree': [0.3, 0.4],
                  'random_state': [42]}

xgb_log = Logger(estimator = 'XGBRegressor',
                 eda_info = EDA_INFO,
                 val_info = '10 блоков, со стандартизацией и PCA', 
                 grid = xgb_param_grid, outfile = r'logs\tree_based_models.txt')

xgb_log.best_params, xgb_log.scores = kosherGridSearchCV(train_df_x, train_df_y, XGBRegressor,
                                                         xgb_param_grid, features_to_standard,
                                                         compare_means, 42, 10)

xgb_log.make_note()

  0%|          | 0/16 [00:00<?, ?it/s]

Точность: 0.13067738612674912 +- 0.027263880480026162 (mean +- std)
Худшая точность: 0.194933948684656
Медианная точность: 0.12786470287346788
Лучшая точность: 0.08671168610621242



In [123]:
xgb_model, xgb_pred = teach_and_predict(train_df, test_df, features_to_standard, XGBRegressor, **xgb_log.best_params)

write_in_file(xgb_pred, r'predictions\xgb_preds.csv')

Скор на отложенной выборке - 0.13710

**LightGBM**

In [149]:
from lightgbm import LGBMRegressor

lgb_param_grid = {'boosting_type': ['gbdt', 'goss'],
                  'max_depth': [5, 6, 7],
                  'learning_rate': [0.05, 0.01],
                  'n_estimators': [150, 250, 500],
                  'subsample': [0.3, 0.4],
                  'colsample_bytree': [0.3, 0.4],
                  'random_state': [42]}

lgb_log = Logger(estimator = 'LGBMRegressor',
                 eda_info = EDA_INFO,
                 val_info = '10 блоков, со стандартизацией и PCA', 
                 grid = xgb_param_grid, outfile = r'logs\tree_based_models.txt')

lgb_log.best_params, lgb_log.scores = kosherGridSearchCV(train_df_x, train_df_y, LGBMRegressor,
                                                         lgb_param_grid, features_to_standard,
                                                         compare_means, 42, 10)

lgb_log.make_note()

  0%|          | 0/144 [00:00<?, ?it/s]

Точность: 0.13569009932119655 +- 0.024715423354863243 (mean +- std)
Худшая точность: 0.18785189556012152
Медианная точность: 0.13621359540738412
Лучшая точность: 0.09072813964127918



In [150]:
lgb_model, lgb_pred = teach_and_predict(train_df, test_df, features_to_standard, LGBMRegressor, **lgb_log.best_params)

write_in_file(lgb_pred, r'predictions\lgb_preds.csv')

Скор на отложенной выборке - 0.13732

In [151]:
from sklearn.ensemble import RandomForestRegressor

rf_param_grid = {'n_estimators': [750, 1000, 1250],
                 'max_depth': [6, 7, 8],
                 'max_features': [0.3, 0.5],
                 'random_state': [42],
                 'max_samples': [0.3, 0.5]}

rf_log = Logger(estimator = 'RandomForestRegressor',
                eda_info = EDA_INFO,
                val_info = '10 блоков, со стандартизацией и PCA', 
                grid = xgb_param_grid, outfile = r'logs\tree_based_models.txt')

rf_log.best_params, rf_log.scores = kosherGridSearchCV(train_df_x, train_df_y, RandomForestRegressor,
                                                       rf_param_grid, features_to_standard,
                                                       compare_means, 42, 10)

rf_log.make_note()

  0%|          | 0/36 [00:00<?, ?it/s]

Точность: 0.14869504050569898 +- 0.02291633520711987 (mean +- std)
Худшая точность: 0.19849275671574707
Медианная точность: 0.1439176853871
Лучшая точность: 0.10571954809850741



In [ ]:
rf_model, rf_pred = teach_and_predict(train_df, test_df, features_to_standard, RandomForestRegressor, **rf_log.best_params)

write_in_file(rf_pred, r'predictions\rf_preds.csv')

Скор на отложенной выборке крайне плох - 0.15959

# Проверка целесообразности использования t-теста стьюдента для сравнения средних скоров

Убедимся на будущее (для сравнения средних значений скоров t-тестом Стьюдента) в том, что скоры распределены нормально, а их дисперсии различаются статистически незначимо. $p_{value} = 0.05$

In [158]:
import scipy

def f_test(x, y):
    x = np.array(x)
    y = np.array(y)
    # По умолчанию numpy считает смещённую оценку выборочной дисперсии. 
    # параметр ddof, приравненый к единице, исправляет это недоразумение 
    f_stat = np.var(x, ddof = 1) / np.var(y, ddof = 1) 
    dfn = x.size - 1 
    dfd = y.size - 1 
    p_value = 1 - scipy.stats.f.cdf(f_stat, dfn, dfd) 
    return f_stat, p_value

Проверка нормальности:

In [176]:
xgb_pvalue = scipy.stats.shapiro(xgb_log.scores)[1]
lgb_pvalue = scipy.stats.shapiro(lgb_log.scores)[1]
rf_pvalue = scipy.stats.shapiro(rf_log.scores)[1]
svr_pvalue = scipy.stats.shapiro(svr_log.scores)[1]
lasso_pvalue = scipy.stats.shapiro(lasso_log.scores)[1]
ridge_pvalue = scipy.stats.shapiro(ridge_log.scores)[1]
elnet_pvalue = scipy.stats.shapiro(elnet_log.scores)[1]

print('Скоры XGBoost распределены нормально:', xgb_pvalue > 0.05, 'pvalue:', xgb_pvalue)
print('Скоры LightGBM распределены нормально:', lgb_pvalue > 0.05, 'pvalue:', lgb_pvalue)
print('Скоры Random Forest распределены нормально:', rf_pvalue > 0.05, 'pvalue:', rf_pvalue)
print('Скоры SVR распределены нормально:', svr_pvalue > 0.05, 'pvalue:', svr_pvalue)
print('Скоры Lasso распределены нормально:', lasso_pvalue > 0.05, 'pvalue:', lasso_pvalue)
print('Скоры Ridge распределены нормально:', ridge_pvalue > 0.05, 'pvalue:', ridge_pvalue)
print('Скоры Elastic Net распределены нормально:', elnet_pvalue > 0.05, 'pvalue:', elnet_pvalue)

Скоры XGBoost распределены нормально: True pvalue: 0.4042729437351227
Скоры LightGBM распределены нормально: True pvalue: 0.7962411046028137
Скоры Random Forest распределены нормально: True pvalue: 0.5831871628761292
Скоры SVR распределены нормально: False pvalue: 0.03177593648433685
Скоры Lasso распределены нормально: False pvalue: 0.0024544140323996544
Скоры Ridge распределены нормально: False pvalue: 0.003651607548817992
Скоры Elastic Net распределены нормально: False pvalue: 0.003158147446811199


Да, тест проводится лишь для лучших скоров каждой модели, но там, где $p_{value}$ больше 0.05, он, помимо всего прочего, значительно больше, чем 0.05, так что оснований считать, что скоры распределены нормально не всегда, нет 

Проверка равенство дисперсий

In [177]:
xgb_lgb_pvalue = f_test(xgb_log.scores, lgb_log.scores)[1]
xgb_rf_pvalue = f_test(xgb_log.scores, rf_log.scores)[1]
lgb_rf_pvalue = f_test(lgb_log.scores, rf_log.scores)[1]
print('Дисперсии XGBoost и LightGBM равны:', xgb_lgb_pvalue > 0.05, 'pvalue:', xgb_lgb_pvalue)
print('Дисперсии XGBoost и Random Forest равны:', xgb_rf_pvalue > 0.05, 'pvalue:', xgb_rf_pvalue)
print('Дисперсии LightGBM и Random Forest равны:', lgb_rf_pvalue > 0.05, 'pvalue:', lgb_rf_pvalue)

Дисперсии XGBoost и LightGBM равны: True pvalue: 0.3873877548496365
Дисперсии XGBoost и Random Forest равны: True pvalue: 0.30654736598608534
Дисперсии LightGBM и Random Forest равны: True pvalue: 0.41276749272113566


Выводы аналогичны

Таким образом, сравнение средних при помощи t-test'а оправдано 